## Product Color Classification

This project makes use of [FastAI](https://fast.ai) library to train a product color classifier on the [Digikala](https://digikala.com) product dataset released as part of their [data science](https://quera.ir/contest/assignments/20120/problems) competition.

In [ ]:
#hide
from fastai.vision.all import *
from PIL import Image

import os
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [ ]:
# variables
train_path = Path("train")
test_path = Path("test")

img_height = img_width = 224
nb_epoch = 50
batch_size = 32
nb_classes = len(os.listdir(training_dir))

In [ ]:
def get_color(file_path): return str(file_path).split("/")[1] 

dls = ImageDataLoaders.from_path_func(train_path, get_image_files(train_path), 
                                      label_func=get_color, seed=42, valid_pct=0.2,
                                      item_tfms=Resize(224))
learn = cnn_learner(dls, resnet34, metrics=error_rate)

learn.fine_tune(1)

In [ ]:
# functions

def plot_training(H, N, plotPath):
    # construct a plot that plots and saves the training history
    plt.style.use("ggplot")
    plt.figure()
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig(plotPath)
    
def show_image(x):
    """show the input image x
    Arguments:
        x {numpy array} -- input image x as a 3-dimensional numpay array
    Returns:
        None
    """

    plt.imshow(x)
    
def load(filename):
    np_image = Image.open(filename)
    np_image = np.array(np_image).astype('float32')/255
    np_image = transform.resize(np_image, (img_height, img_width, 3))
    np_image = np.expand_dims(np_image, axis=0)
    return np_image

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20)
train_generator = train_datagen.flow_from_directory(training_dir, target_size=(img_height, img_width), batch_size=batch_size)

# test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = validation_datagen.flow_from_directory(test_dir,target_size=(img_height, img_width),batch_size=batch_size)

# load the VGG16 network, ensuring the head FC layer sets are left
# off

baseModel = MobileNetV2(weights="imagenet", include_top=False, input_shape=(img_height, img_width, 3))

for layer in baseModel.layers:
    layer.trainable = False

# construct the head of the model that will be placed on top of the
# the base model
# x=baseModel.output
# x=GlobalAveragePooling2D()(x)
# x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
# x=Dense(1024,activation='relu')(x) #dense layer 2
# x=Dense(512,activation='relu')(x) #dense layer 3
# preds=Dense(nb_classes,activation='softmax')(x) #final layer with softmax activation

# model = Model(inputs=baseModel.input, outputs=preds)

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(nb_classes, activation="softmax",activity_regularizer=L2(0.01))(headModel)
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

model.compile(loss="categorical_crossentropy", optimizer='adamax', metrics=[Precision()])

print("[INFO] training head...")
H = model.fit(train_generator,
              steps_per_epoch=totalTrain // batch_size,
              epochs=nb_epoch)

In [ ]:
train_generator.class_indices.keys()

In [ ]:
# prediction

decode_mapping = ['black', 'blue', 'brown', 'green', 'grey', 'orange', 'pink', 'purple', 'red', 'silver', 'white', 'yellow']

encode_mapping = {'pink':1,
           'purple':2,
           'yellow':3,
           'orange':4,
           'white':5,
           'silver':6,
           'grey':7,
           'black':8,
           'red':9,
           'brown':10,
           'green':11,
           'blue':12,
}

results = []
for file in os.listdir(test_dir):
    dic = {}
    img = load(test_dir+file)
    y_prob = model.predict(img)
    y_classes = decode_mapping[y_prob.argmax(axis=-1)[0]]
    dic['file_name'] = file
    dic['color_id'] = encode_mapping[y_classes]
    results.append(dic)

In [ ]:
pd.DataFrame(results).to_csv("results_q4.csv",index=False)